In [166]:
import requests                         # for sending HTTP requests

from tqdm.notebook import tqdm, trange  # for progress bars
from selenium import webdriver
from scrapy import Selector             # for parsing HTML content
import pandas as pd
import time
import json
import os

In [167]:

def fetch_page(url, delay=1):
  """
  """
  headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
  }
  time.sleep(delay)
  try:
    response = requests.get(url, timeout=None, headers= headers) 
  except Exception as e:
    print('Error', e)
    pass
  if not response.ok:
    print("Something went wrong", response.status_code)
    pass
  html = response.content
  
  return Selector(text=html)

In [168]:
# def extract_all_listings(page_sel): 
#   """
#   Inputs: 
#     page_sel: a Selector object which contains 
#       the contents of the Zillow website of houses
#   Outputs:
#     rental_list: a list of listings contained within page_sel
#   """
#   objects = page_sel.xpath('//div[contains(@class, "StyledPropertyCardDataWrapper")]')
#   return objects


In [169]:
# Hide this cell
# def get_listing_info(listing_sel):
#   """
#   Inputs:
#     listing_sel: a Selector object which contains the content
#       of one listing
#   Outputs:
#     unit_dict: a dictionary which contains price, number of bedrooms, 
#       number of bathrooms, sqft, postal code
#   """
#   address = listing_sel.css("a ::text").get()
#   zip_code = address[-5:]
#   price = listing_sel.css("div span ::text").get()[1:]
#   bbs = listing_sel.css("ul li b ::text").getall()
#   bed = bbs[0]
#   bath = bbs[1]
#   sqft = bbs[2]
#   town = address.split(',')[1]
#   address = address.split(',')[0]
#   return {'address': address, 'zip_code': zip_code, 'price': price, 'num_bedrooms': bed, 
#           'num_bathrooms': bath, 'sqft': sqft, 'town': town}

In [170]:

# objects = extract_all_listings(sel)
# # print(sel)
# info_list = [get_listing_info(x) for x in objects]
# info_df = pd.DataFrame(info_list)
# display(info_df)
# # display(info_df.info)
# display(sel.css('div#grid-search-results').get())

In [171]:
# footer = sel.xpath('//ul[contains(@class, "PaginationList")]/li[contains(@class, "PaginationJumpItem")]').css('::attr(href)')[1]
# print(footer)


In [172]:
# footer_list = [get_listing_info(foot) for foot in footer]
# display(footer_list)
# print(len(footer_list))
# print(footer_list[30])
# df = pd.DataFrame(footer_list)
# display(df)
# print(df.info())

In [173]:
# print(df.nunique())

In [174]:
# def recursive_scrape(url, list_so_far):
#   if len(list_so_far) > 30:
#     return list_so_far
#   if url[0] != '/':
#     return list_so_far
#   url_modified = "https://www.zillow.com" + url
#   sel = fetch_page(url_modified)
#   objects = extract_all_listings(sel)
#   info_list = [get_listing_info(x) for x in objects]
#   list_so_far.append(info_list)
#   footer = sel.xpath('//ul[contains(@class, "PaginationList")]/li[contains(@class, "PaginationJumpItem")]').css('::attr(href)').getall()[1]
#   return recursive_scrape(footer, list_so_far)
# agg_list = recursive_scrape('/boston-ma', [])

  
  
  

In [175]:

def fetch_by_script(full_page_selector):
  """
  """
  #find the script which loads in the page with data
  script_text = full_page_selector.css('script#__NEXT_DATA__').get()
  no_first = script_text.split('>')[1]
  no_last = no_first.split('<')[0]
  listing_json = json.loads(no_last)
  cat_1 = listing_json['props']['pageProps']['searchPageState']['cat1']['searchResults']['listResults']
  page_df = pd.DataFrame(cat_1)
  return(page_df)

selector = fetch_page('https://www.zillow.com/boston-ma')
# display(fetch_by_script(selector))


# print(cat_1)


#the zpid is the same as id, so dropping zpid
#making a new dataframe with only the columns we want


In [177]:
import os
def rec_fetch_all_pages(url):
  full_url = 'https://www.zillow.com' + url
  sel = fetch_page(full_url)
  wanted_df = fetch_by_script(sel)
  #i used AI to save the df to a json in my data folder and check to make sure file exists
  if wanted_df is not None:
    output_dir = '../data/raw/zillow'
    if not os.path.exists(output_dir):
      os.makedirs(output_dir)
    file_path = os.path.join(output_dir, f'zillow_{url.replace("/","")}.json')
    wanted_df.to_json(file_path, orient='records', lines=True)
    print(f'DataFrame saved to {file_path}')
     
  else:
    print(f'No data found for {url}')
  footer = sel.xpath('//ul[contains(@class, "PaginationList")]/li[contains(@class, "PaginationJumpItem")]').css('::attr(href)').getall()[1]
  print(footer)
  if footer is None:
    return
  #basically, zillow only lets you get 20 pages, and the next arrow would point
  #to the 20th page even if it should point to 21st
  if footer == url:
    return
  return rec_fetch_all_pages(footer)

rec_fetch_all_pages('/boston-ma/')

DataFrame saved to ../data/raw/zillow/zillow_boston-ma.json
/boston-ma/2_p/
DataFrame saved to ../data/raw/zillow/zillow_boston-ma2_p.json
/boston-ma/3_p/
DataFrame saved to ../data/raw/zillow/zillow_boston-ma3_p.json
/boston-ma/4_p/
DataFrame saved to ../data/raw/zillow/zillow_boston-ma4_p.json
/boston-ma/5_p/
DataFrame saved to ../data/raw/zillow/zillow_boston-ma5_p.json
/boston-ma/6_p/
DataFrame saved to ../data/raw/zillow/zillow_boston-ma6_p.json
/boston-ma/7_p/
DataFrame saved to ../data/raw/zillow/zillow_boston-ma7_p.json
/boston-ma/8_p/
DataFrame saved to ../data/raw/zillow/zillow_boston-ma8_p.json
/boston-ma/9_p/
DataFrame saved to ../data/raw/zillow/zillow_boston-ma9_p.json
/boston-ma/10_p/
DataFrame saved to ../data/raw/zillow/zillow_boston-ma10_p.json
/boston-ma/11_p/
DataFrame saved to ../data/raw/zillow/zillow_boston-ma11_p.json
/boston-ma/12_p/
DataFrame saved to ../data/raw/zillow/zillow_boston-ma12_p.json
/boston-ma/13_p/
DataFrame saved to ../data/raw/zillow/zillow_bos